# Lab 10: Feature Engineering & Cross-Validation
** This assignment is due 04/04/2018 at 11:59pm (graded on accuracy) **

In this lab, you will practice using scikit-learn to do feature engineering and cross-validation to produce a model with low error on held-out data.

In [ ]:
# Run this cell to set up your notebook
import seaborn as sns
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()
sns.set_context("talk")

from IPython.display import display, Latex, Markdown

### Introduction

For this lab, we will use a toy dataset to predict the house prices in Boston with data provided by the `sklearn.datasets` package.

Run the following cell to load the data. This will return a dictionary object which includes keys for:
    - `data` : the covariates (X)
    - `target` : the response vector (Y)
    - `feature_names`: the column names
    - `DESCR` : a full description of the data

In [ ]:
from sklearn.datasets import load_boston

boston_data = load_boston()
print(boston_data.keys())

In [ ]:
print(boston_data['DESCR'])

A look at the `DESCR` attribute tells us the data contains these features:

    1. CRIM      per capita crime rate by town
    2. ZN        proportion of residential land zoned for lots over 
                 25,000 sq.ft.
    3. INDUS     proportion of non-retail business acres per town
    4. CHAS      Charles River dummy variable (= 1 if tract bounds 
                 river; 0 otherwise)
    5. NOX       nitric oxides concentration (parts per 10 million)
    6. RM        average number of rooms per dwelling
    7. AGE       proportion of owner-occupied units built prior to 1940
    8. DIS       weighted distances to five Boston employment centres
    9. RAD       index of accessibility to radial highways
    10. TAX      full-value property-tax rate per 10,000 USD
    11. PTRATIO  pupil-teacher ratio by town
    12. B        1000(Bk - 0.63)^2 where Bk is the proportion of black 
                 residents by town
    13. LSTAT    % lower status of the population
    
Let's now convert this data into a pandas DataFrame. 

In [ ]:
boston = pd.DataFrame(boston_data['data'], columns=boston_data['feature_names'])
boston.head()

### Question 1

Let's model this housing price data! Before we can do this, however, we need to split the data into training and test sets. The latter, held-out points will be used to choose the best performing model. Remember that the response vector (housing prices) lives in the `target` attribute.

Use the [`train_test_split`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to split out 10% of the data for test. Call the resulting splits `X_train`, `X_test`, `Y_train`, `Y_test`.

In [ ]:
from sklearn.model_selection import train_test_split
np.random.seed(47)

X = boston
Y = pd.Series(boston_data['target'])

X_train, X_test, Y_train, Y_test = ...,...,...,...
### BEGIN SOLUTION
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.10)
### END SOLUTION
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

In [ ]:
assert X_train.shape == (455, 13)
assert X_test.shape == (51, 13)
assert Y_train.shape == (455, )
assert Y_test.shape == (51, )

### Question 2

As a warmup, fit a linear model to describe the relationship between the housing price and all available covariates. We've imported `sklearn.linear_model` as lm, so you can use that instead of typing out the whole module name. Running the cell should create a scatter plot for our predictions vs the true prices.

In [ ]:
import sklearn.linear_model as lm

linear_model = lm.LinearRegression()


# Fit your classifier
#linear_model.fit(...)

### BEGIN SOLUTION
linear_model.fit(X_train, Y_train)
### END SOLUTION


In [ ]:

# Predict housing prices on the test set
Y_pred = ... # linear_model.predict(...)


### BEGIN SOLUTION
Y_pred = linear_model.predict(X_test)
### END SOLUTION

# Plot predicted vs true prices
plt.scatter(Y_test, Y_pred, alpha=0.5)
plt.xlabel("Prices")
plt.ylabel("Predicted prices")
plt.title("Prices vs Predicted prices")

### Question 3

As we find from the scatter plot, our model is not perfect. If it were perfect, we would see the identity line (i.e. a line of slope 1). Compute the root mean squared error (RMSE) of the predicted responses: 

$$
\textbf{RMSE} = \sqrt{\frac{1}{n}\sum_{i=1}^n \left( y_i - \hat{y}_i \right)^2 }
$$

Fill out the function below and compute the RMSE for our predictions on both the training data `X_train` and the test set `X_test`.  Note your implementation should not contain the word **"for"** (...that would be very slow).

In [ ]:
def rmse(actual_y, predicted_y):
    """
    Args:
        predicted_y: an array of the prediction from the model
        actual_y: an array of the groudtruth label
        
    Returns:
        The root mean square error between the prediction and the groudtruth
    """
### BEGIN SOLUTION
    return np.sqrt(np.mean((actual_y - predicted_y) ** 2))
### END SOLUTION
    return ...

In [ ]:
train_error = ...
test_error = ...

### BEGIN SOLUTION
train_error = rmse(Y_train, linear_model.predict(X_train))
test_error = rmse(Y_test, Y_pred)
### END SOLUTION

print("Training RMSE:", train_error)
print("Test RMSE:", test_error)

In [ ]:
assert np.allclose((train_error, test_error), (4.56291225689, 5.88492861688))

## Cross Validation

**Warning**: don't use the test set to perform the feature selection! It may lead to over-fitting. We want to avoid using the test set too frequently. When selecting features or choosing hyper-parameters, we can split the training set further into train and validation sets. Then we can use the validation error to help select hyper-parameters.

Try $k$-fold cross-validation to select the best subset of features for our model. Recall the approach looks something like:

<img src="cv.png" width=500px>

### Question 4

Scikit-learn has built-in support for cross validation.  However, to better understand how cross validation works complete the following function which cross validates a given model.

1. Use the [`KFold.split`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) function to get 4 splits on the training data. Note that `split` returns the indices of the data for that split.
2. For each split, select out the rows and columns based on the split indices and features.
3. Compute the RMSE on the validation split.
4. Return the average error across all cross validation splits.


In [ ]:
from sklearn.model_selection import KFold

def compute_CV_error(model, X_train, Y_train):
    '''
    Split the training data into 4 subsets.
    For each subset, 
        fit a model holding out that subset
        compute the MSE on that subset (the validation set)
    You should be fitting 4 models total.
    Return the average MSE of these 4 folds.

    Args:
        model: an sklearn model with fit and predict functions 
        X_train (data_frame): Training data
        Y_train (data_frame): Label 

    Return:
        the average validation MSE for the 4 splits.
    '''
    kf = KFold(n_splits=4)
    validation_errors = []
    
    for train_idx, valid_idx in kf.split(X_train):
        # split the data
        split_X_train, split_X_valid = ..., ...
        split_Y_train, split_Y_valid = ..., ...

        # Fit the model on the training split
        ...
        
        # Compute the RMSE on the validation split
        error = ...

        ### BEGIN SOLUTION
        # split the data
        split_X_train, split_X_valid = X_train.iloc[train_idx], X_train.iloc[valid_idx]
        split_Y_train, split_Y_valid = Y_train.iloc[train_idx], Y_train.iloc[valid_idx]
      
        # Fit the model on the training split
        model.fit(split_X_train, split_Y_train)
        
        error = rmse(model.predict(split_X_valid), split_Y_valid)
        ### END SOLUTION
        
        validation_errors.append(error)
        
    return np.mean(validation_errors)

In [ ]:
assert np.isclose(
    compute_CV_error(lm.LinearRegression(), X_train[['TAX', 'INDUS', 'CRIM']],Y_train),
    7.5974094557701459)

### Question 5

Use the above cross validation function to determine which of the following feature sets to use:

In [ ]:
feature_sets = [
    ['TAX', 'INDUS', 'CRIM'], 
    ['RM', 'LSTAT', 'PTRATIO'], 
    ['RM', 'B', 'NOX'], 
    ['TAX', 'LSTAT', 'DIS']
]

errors = []
for feat in feature_sets:
    print("Trying features:", feat)
    model = lm.LinearRegression()
    error = ... # compute the cross validation error
    print("\tRMSE:", error)
    errors.append(error)

best_err_idx = ...
best_err = ...
best_feature_set = ...

### BEGIN SOLUTION
print("\n\nBegin Solution")
errors = []
for feat in feature_sets:
    print("Trying features:", feat)
    model = lm.LinearRegression()
    error = compute_CV_error(model, X_train[feat],Y_train)
    print("\tRMSE:", error)
    errors.append(error)    

best_err_idx = np.argmin(errors)
best_err = errors[best_err_idx]
best_feature_set = feature_sets[best_err_idx]
### END SOLUTION

for i in range(4):
    print('{}, error: {}'.format(feature_sets[i], errors[i]))

best_feature_set, best_err

In [ ]:
assert best_feature_set == ['RM', 'LSTAT', 'PTRATIO']
assert np.isclose(best_err, 5.221575997721903)

### Question 6
Finally, fit a linear classifier using your best feature set and predict housing prices for your original test set. Compute the final MSE.

In [ ]:
# Fit your classifier
...

# Predict points from our test set and calculate the mse
train_rmse = ... 
test_rmse = ...


### BEGIN SOLUTION
model = lm.LinearRegression()
model.fit(X_train[best_feature_set], Y_train)
train_rmse = rmse(model.predict(X_train[best_feature_set]), Y_train) 
test_rmse = rmse(model.predict(X_test[best_feature_set]), Y_test)
### END SOLUTION

print("Train RMSE", train_rmse)
print("KFold Validation RMSE", best_err)
print("Test RMSE", test_rmse)

Notice that the test error is higher than the validation error which is higher than the training error.  Why is this the case?

In [ ]:
assert np.abs(test_rmse - 5.846401452163672) < 1e-3

Nice! You've used $k$-fold cross-validation to fit a linear regression model to the housing data.

In the future, you'd probably want to use something like [`cross_val_predict`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html) to automatically perform cross-validation, but it's instructive to do it yourself at least once.

## Submission

Congrats! You are finished with this assignment. Please don't forget to submit by 11:59 pm!